Code for manipulating Turing machines

In [7]:
import copy

def initialConf (machine, word) :
    return {
        'state' : machine['initial'],
        'input tape' : word,
        'input head' : 0,
        'work tape' : [''],
        'work head' : 0
    }
        

def doesTransitionApply (trans, conf) :
    
    #wrong state
    if trans['state before'] != conf['state']:
        return False
    
    #wrong input letter
    markedtape = ['⊢']+ conf['input tape'] + ['⊣']
    if ('input letter before' in trans) and (trans['input letter before'] != markedtape[conf['input head']]):
        return False
    
    #wrong work letter
    if ('work letter before' in trans) and (trans['work letter before'] != (conf['work tape'])[conf['work head']]):
        return False
    
    #input head falls out of input tape
    if ('move input head' in trans) and ((conf['input head'] + trans['move input head'] >= len(markedtape)) 
                                         or (conf['input head'] + trans['move input head'] < 0)):
        return False

    #work head falls out of work tape to the left
    if ('move work head' in trans) and (trans['move work head'] + conf['work head'] < 0):
        return False
    
    return True

#applies a transitions and returns the new configuration
def applyTransition(trans,conf):
    if not doesTransitionApply(trans,conf):
        raise Exception('tried to apply transition that does not apply')
    
    if 'halt' in trans:
        return trans['halt']
    
    newconf = copy.deepcopy(conf);
    
    newconf['state'] = trans['state after']
    
    if 'move input head' in trans:
        newconf['input head']+=trans['move input head']
    
    if 'work tape after' in trans:
        newconf['work tape'][newconf['work head']] = trans['work tape after']

    if 'move work head' in trans:
        newconf['work head']+=trans['move work head']
            
    #if the work head moved out of work tape, add a new blank symbol
    if newconf['work head'] >= len(newconf['work tape']):
        newconf['work tape'] += ['']

    return newconf

#returns the list of all avaialable transitions
def availableTransitions(machine, conf):

    retval = []

    if (conf == 'accept') or (conf == 'reject'):
        return retval

    for t in machine['transitions']:
        if doesTransitionApply(t, conf):
            retval += [t]
    return retval

#returns the list of all configurations in a run, for the given input string
#if several transitions apply, the first one is used

def run(machine, string):
    conf = initialConf(machine, string)

    #the return value is a list of configurations
    retval = []
    while True:
        retval += [conf]
        
        if conf == 'accept' or conf == 'reject':
            break
            
        transitionList = availableTransitions(machine, conf);
        if len(transitionList) == 0:
            retval+=['reject']
            break
        
        #we use first available transition
        t =transitionList[0]
        conf = applyTransition(t,conf)
        
    return retval

from IPython.core.display import HTML

def cellHTML(content, head) :
    if head:
        return '<span style="background-color:red; padding: 10px; margin: 5px">' +  content + '</span>'
    else:
        return '<span style="background-color:lightgrey; padding: 10px; margin: 5px">' +  content + '</span>'
    
def tapeHTML(string, head) :
    if (head < 0 or head >= len(string)):
        raise Exception('head out of bounds')
    index = 0
    retval = '<span style="margin:20px">'
    for x in string:
        retval += cellHTML(x, index == head)
        index += 1
    return retval + '</span>'


def confHTML(conf) :
    retval = '<div style="padding:20px">'
    if (conf == 'accept' or conf == 'reject'):
        retval += conf
    else:
        retval +='<span style="display:inline-block; width:100px"> state:' + conf['state'] + '</span>'
        retval += 'input tape:'
        retval += tapeHTML(['⊢']+ conf['input tape'] + ['⊣'], conf['input head'])
        retval += 'work tape:'
        retval += tapeHTML(conf['work tape'], conf['work head'])
    
    retval += '</div>'
    return retval 

def displayConf(conf) :
    HTML(confHTML(conf))
    
def displayRun(machine,string):
    retval = ''
    for conf in run(machine, string):
        retval += confHTML(conf)
    return HTML(retval)

# Palindromes

Here is a Turing machine with input alphabet ['a','b'] that accepts exactly the palindromes.

In [4]:
machine = { 'initial' : 'p',
          'transitions' : [
           {'state before' : 'p', 'state after' : 'p', 'input letter before' : '⊢','move input head' : 1,'move work head' : 1,'work tape after' : '⊢'},
              {'state before' : 'p', 'state after' : 'p', 'input letter before' : 'a', 'move input head' : 1, 'move work head' : 1,'work tape after' : 'a'},
              {'state before' : 'p', 'state after' : 'p', 'input letter before' : 'b', 'move input head' : 1, 'move work head' : 1, 'work tape after' : 'b'},
              {'state before' : 'p', 'state after' : 'q', 'input letter before' : '⊣', 'move input head' : -1},
              {'state before' : 'q', 'state after' : 'q', 'input letter before' : 'a','move input head' : -1},
              {'state before' : 'q', 'state after' : 'q', 'input letter before' : 'b', 'move input head' : -1},
              {'state before' : 'q', 'state after' : 'r', 'input letter before' : '⊢','move input head' : 1,'move work head' : -1},
              {'state before' : 'r', 'state after' : 'r', 'input letter before' : 'a','work letter before' : 'a' ,'move work head' : -1, 'move input head' : 1},
              {'state before' : 'r', 'state after' : 'r', 'input letter before' : 'b','work letter before' : 'b' ,'move work head' : -1, 'move input head' : 1},
            {'state before' : 'r', 'input letter before' : '⊣','halt' : 'accept' },
           ]
          }

In [5]:
displayRun(machine,[])